### Use $\color{blue}{\text{python 2.7}}$ for this guide

##### Use the SCOBRA module
Scobra has dependencies of cobra, pandas, matplotlib, numpy, scipy, lxml, xlwt, xlrd

##### Import Modules

In [1]:
import scobra
import csv
import pickle
from cobra import Reaction
import numpy as np
import pandas as pd

##### Load the Model (excel version)

In [2]:
#m=scobra.Model(< /Path/ of / the / Model file.xls >)
m=scobra.Model('setaria_c4x4_v12a.xls')

WATER_CP added to metabolite list in H2O_tx_BS_base in row 5038
MG+2_CP added to metabolite list in Mg_tx_BS_base in row 6290
AMMONIUM_CP added to metabolite list in NH4_tx_BS_base in row 6498
NITRATE_CP added to metabolite list in NO3_tx_BS_base in row 6594
Pi_CP added to metabolite list in PHOSPHATE_tx_BS_base in row 7082
PROTON_CP added to metabolite list in PROTON_tx_BS_base in row 7258
SULFATE_CP added to metabolite list in SO4_tx_BS_base in row 18470
L-ALPHA-ALANINE_CP added to metabolite list in ALA_CP_BS_lwr_rev_transport in row 19490
ARG_CP added to metabolite list in ARG_CP_BS_lwr_rev_transport in row 19498
ASN_CP added to metabolite list in ASN_CP_BS_lwr_rev_transport in row 19506
L-ASPARTATE_CP added to metabolite list in ASP_CP_BS_lwr_rev_transport in row 19514
GLN_CP added to metabolite list in GLN_CP_BS_lwr_rev_transport in row 19522
GLT_CP added to metabolite list in GLT_CP_BS_lwr_rev_transport in row 19530
GLY_CP added to metabolite list in GLY_CP_BS_lwr_rev_transport 

#### Defining a few functions

In [3]:
def SplitRevRxn(model):
    #modify.convert_to_irreversible(model)
    reactions_to_add = []
    for reaction in model.reactions:
        if reaction.reversibility == True:
            reverse_reaction = Reaction(reaction.id + "_reverse")
            reverse_reaction.lower_bound = min(0, reaction.upper_bound) * -1
            reverse_reaction.upper_bound = reaction.lower_bound * -1
            reaction.lower_bound = 0
            reaction.upper_bound = max(0, reaction.upper_bound)
            #Make the directions aware of each other
            reaction.reflection = reverse_reaction
            reverse_reaction.reflection = reaction
            reaction_dict = dict([(k, v*-1) for k, v in reaction._metabolites.items()])
            reverse_reaction.add_metabolites(reaction_dict)
            reverse_reaction._model = reaction._model
            reverse_reaction._genes = reaction._genes
            for gene in reaction._genes:
                gene._reaction.add(reverse_reaction)
            reverse_reaction._gene_reaction_rule = reaction._gene_reaction_rule
            reactions_to_add.append(reverse_reaction)
    model.add_reactions(reactions_to_add)
    return model

def MakeInvGEWtDict(m, DWt='Mean',
                GEbase="GSMx2rxn_baseGE_N0_edited.txt", 
                GElwr="GSMx2rxn_lwrGE_N0_edited.txt", 
                GEmid="GSMx2rxn_midGE_N0_edited.txt", 
                GEtip="GSMx2rxn_tipGE_N0_edited.txt"):
    '''m= Scobra Model File,
    GEfiles= give GE files path/ name
    DWt='Mean' or '10Mean' or 0.1Mean or 'Any Other Number'
    '''
    
    M=m.copy()
    #######SPLITTING REVERSIBLE REACTIONS
    M_split=M.copy()
    M_split=SplitRevRxn(M_split)
    
    ##making dict of GE values    
    b1=open(GEbase,'r')
    b2=[x.strip() for x in b1.readlines()]
    b3=[y.split('\t') for y in b2]
    baseGE = {(b[0]+'_base'):(1/float(b[1])) for b in b3}
    b1.close()
    #print(baseGE)
    
    l1=open(GElwr,'r')
    l2=[x.strip() for x in l1.readlines()]
    l3=[y.split('\t') for y in l2]
    lwrGE = {(l[0]+'_lwr'):(1/float(l[1])) for l in l3}
    l1.close()
    #print(lwrGE)
    
    z1=open(GEmid,'r')
    z2=[x.strip() for x in z1.readlines()]
    z3=[y.split('\t') for y in z2]
    midGE = {(z[0]+'_mid'):(1/float(z[1])) for z in z3}
    z1.close()
    #print(midGE)
    
    t1=open(GEtip,'r')
    t2=[x.strip() for x in t1.readlines()]
    t3=[y.split('\t') for y in t2]
    tipGE = {(t[0]+'_tip'):(1/float(t[1])) for t in t3}
    t1.close()
    #print(tipGE)
    
    allGE={}
    allGE.update(baseGE)
    allGE.update(lwrGE)
    allGE.update(midGE)
    allGE.update(tipGE)
    
    #adding _reverse reactions in GE dict
    revrxn=[r for r in M_split.Reactions() if "_reverse" in r]
    revtemp=[x.replace("_reverse","") for x in revrxn]
    revGE={v+"_reverse":allGE[v] for v in revtemp if v in allGE.keys()}
    allGE.update(revGE)
    
    rest=[r for r in M_split.Reactions() if r not in allGE.keys()]
    NOGE=[]
    for rx in rest:
        if '_CP_BS' in rx or '_MPBS' in rx:
            NOGE.append(rx)
        if '_tx' in rx or '_biomass' in rx:
            NOGE.append(rx)
        if 'Lipids' in rx or 'AminoAcid' in rx or 'NuclicAcid' in rx or 'CarbosLignin' in rx:
            NOGE.append(rx)
    
    y=[1/Val for Val in allGE.values()]
    if DWt == 'Mean':
        YV=1/np.mean(y)
        restGE={x:YV for x in rest if x not in NOGE}
    elif DWt == '10Mean':
        YV=10/np.mean(y)
        restGE={x:YV for x in rest if x not in NOGE}
    elif DWt == '0.1Mean':
        YV=0.1/np.mean(y)
        restGE={x:YV for x in rest if x not in NOGE}
    elif type(DWt) == int or type(DWt) == float:
        YV=DWt
        restGE={x:YV for x in rest if x not in NOGE}
    else:
        YV=eval(DWt)
        restGE={x:YV for x in rest if x not in NOGE}
    allGE.update(restGE)
    
    return allGE

def SetPhoton(m, ll):
    '''m=scobra model (to set light upper limit)
        ll= lifgt upper limit float value'''
    ll=ll
    ##Light_fix
    P={i:(0,ll) for i in m.Reactions() if 'Photon_tx' in i}
    m.SetConstraints(P)
    #return m

def SetOpenBiomass(m):
    biomass = {x:(-100.0, -0.0) for x in m.Reactions() if '_biomass' in x}
    #Set BIOMASS constraint
    m.SetConstraints(biomass)
    #return m

def SetMaintenance(m):
    mentainanceATP={i:(-8.5,-8.5) for i in m.Reactions() if 'ATPase_tx' in i}
    mentainanceNADP={i:(-0.944,-0.944) for i in m.Reactions() if 'NADPHoxc_tx' in i or 'NADPHoxm_tx' in i or 'NADPHoxp_tx' in i }
    NADPHoxx={i:(0,0) for i in m.Reactions() if 'NADPHoxx_tx' in i}
    
    #Set Mentainance constraint
    m.SetConstraints(mentainanceATP)
    m.SetConstraints(mentainanceNADP)
    #BLOCK NADPHoxx_tx
    m.SetConstraints(NADPHoxx)
    #return m

def SetEQUALPhoton(m):
    P={i:1 for i in m.Reactions() if 'Photon_tx' in i}
    m.SetReacsFixedRatio(P)
    #return m
    
def SetMPBSRatio(m):
    m.SetReacsFixedRatio({'Cell_biomass_MP_tip':1,'Cell_biomass_BS_tip':1})
    m.SetReacsFixedRatio({'Cell_biomass_MP_mid':1,'Cell_biomass_BS_mid':1})
    m.SetReacsFixedRatio({'Cell_biomass_MP_lwr':1,'Cell_biomass_BS_lwr':1})
    m.SetReacsFixedRatio({'Cell_biomass_MP_base':1,'Cell_biomass_BS_base':1})

def SetMPRubiscoRatio(m):
    m.SetReacsFixedRatio({'RIBULOSE-BISPHOSPHATE-CARBOXYLASE-RXN_p_MP_base':3,'RXN-961_p_MP_base':1})
    m.SetReacsFixedRatio({'RIBULOSE-BISPHOSPHATE-CARBOXYLASE-RXN_p_MP_lwr':3,'RXN-961_p_MP_lwr':1})
    m.SetReacsFixedRatio({'RIBULOSE-BISPHOSPHATE-CARBOXYLASE-RXN_p_MP_mid':3,'RXN-961_p_MP_mid':1})
    m.SetReacsFixedRatio({'RIBULOSE-BISPHOSPHATE-CARBOXYLASE-RXN_p_MP_tip':3,'RXN-961_p_MP_tip':1})

def SetBIOMASSRatio(m, r_tmlb=[]):
    '''m=scobra model to set biomas ratio,
    r_tmlb = LIST of ratio values, tip , mid, lwr, base, IN THIS ORDER'''
    D=r_tmlb
    T,M,L,B=D[0],D[1],D[2],D[3]
    m.SetReacsFixedRatio({'Cell_biomass_BS_tip':T,'Cell_biomass_BS_mid':M,'Cell_biomass_BS_lwr':L,'Cell_biomass_BS_base':B})


def BlockExtraRxns(m, rxnlist=[]):
    blklist={i:(0,0) for i in rxnlist}
    m.SetConstraints(blklist)


In [4]:
m4=m.copy() #copying the model

##### Set Specified constraints

In [5]:
SetPhoton(m4, 200.0)
SetOpenBiomass(m4)
SetMaintenance(m4)
rxns2blk=[x for x in m4.Reactions() if 'Plastoquinol_Oxidase_p' in x or 'NADPH_Dehydrogenase_p' in x or 'MALIC-NADP-RXN_c' in x or 'PEPCARBOXYKIN-RXN_c' in x]
BlockExtraRxns(m4, rxns2blk)
adlrxns2blk=[x for x in m4.Reactions() if 'CO2_tx_BS' in x or 'O2_tx_BS' in x or '4.1.1.32-RXN_c_BS' in x or '1.1.1.39-RXN_m_BS' in x]
BlockExtraRxns(m4, adlrxns2blk)

In [6]:
M_split=m4.copy()
M_split=SplitRevRxn(M_split)
M_split1=M_split.copy()
SetEQUALPhoton(M_split1)
SetBIOMASSRatio(M_split1, r_tmlb=[1.283,1.2334,1.9372,5.488]) # Explanation of this Ratio is given in supplementary file
SetMPBSRatio(M_split1)
SetMPRubiscoRatio(M_split1)

##### Set Model Objective
$\color{red}{\text{(Biomass Maximization without integration of Gene-Expression values)}}$

In [7]:
Rxnbiomass=[x for x in M_split1.Reactions() if '_biomass' in x]
M_split1.SetObjective(Rxnbiomass)
M_split1.SetObjDirec(direc='Min') #AS BIOMASS FLUX IS -ve, MINIMIZING THE FLUX is MAXIMIZING BIOMASS

##### Solve the linear equations to fins optimal solution

In [8]:
M_split1.MinFluxSolve()

optimal


##### Obtain the solution of the simulation

In [9]:
SolwoGEBioMax=M_split1.GetSol()

##### Extract the solution in a csv file

In [10]:
with open('BioMax_woGE_Sol.csv', 'wb') as f:
    for key in SolwoGEBioMax.keys():
        f.write("%s, %s\n" %(key, SolwoGEBioMax[key]))

### Simulation with integration of Gene-expression values

##### Set specific constraints
(Biomass values taken from previous BioMax simulation)

In [11]:
M_split2=M_split.copy()
    
SetPhoton(M_split2, 1000.0)
SetEQUALPhoton(M_split2)
SetBIOMASSRatio(M_split2, r_tmlb=[1.283,1.2334,1.9372,5.488])
SetMPBSRatio(M_split2)
SetMPRubiscoRatio(M_split2)
BMax=SolwoGEBioMax.Filter('Cell_biomass_BS_tip')
Bvalue=BMax['Cell_biomass_BS_tip']
BStip_biomass = {x:(Bvalue, Bvalue) for x in M_split2.Reactions() if 'Cell_biomass_BS_tip' in x}
M_split2.SetConstraints(BStip_biomass)

##### Make Gene-Expression Weight dictionary

In [12]:
allGE=MakeInvGEWtDict(M_split2, DWt='Mean',
                      GEbase="GSMx2rxn_baseGE_N0_edited.txt",
                      GElwr="GSMx2rxn_lwrGE_N0_edited.txt",
                      GEmid="GSMx2rxn_midGE_N0_edited.txt",
                      GEtip="GSMx2rxn_tipGE_N0_edited.txt")

# The weight values for each reactions were given in the supplementary file, 
# which can be directly loaded into a dictionary

##### Set Model objective and solve
$\color{red}{\text{(with integration of Gene-Expression values as weight)}}$

Although there is no direct correlation between the expression of an
enzymatic gene and the fluxes through corresponding reaction,
\underline{higher expression} of an enzyme will leads to \underline{better
availability} of the same, thus fecilitates the occurance of
corresponding reaction. Whereas in the case of \underline{lower expression},
\underline{availabilty} of the corresponding enzyme for the reaction is much
more \underline{restricted}. In case of multi-enzymatic reactions, if two or
more genes are essential for a certain reaction (denoted by 'AND' rule
in GPR in model), the enzyme with \textbf{Lowest} expression
(availability) will be the limiting factor. Whereas, if multiple enzymes
can work in parallel to complete a reaction (denoted by 'OR' rule in GPR
in model), the enzyme with \textbf{Highest} expression (availability)
will be most likely to perticipate. So we have used the \textbf{inverse
of the Gene-Expression values as the weight factor} for the objective
function with \underline{minimization} as the goal, thus reactions with
\underline{highly expressed} genes will have a \underline{lower weight} and be
\underline{prefered} for the solution. In opposite, reactions with
\underline{low-expressing genes} will have a \underline{cost (high weight)}
associated with them (\underline{unfavourable}) according to the expression
level of the corresponding enzyme.

In [13]:
M_split2.SetObjective(allGE)
M_split2.SetObjDirec(direc='Min')
    
M_split2.MinFluxSolve()

optimal


##### Extract and print the solution in a file

In [14]:
SolwGE2=M_split2.GetSol()
with open('MinFlx_wGE_Sol.csv', 'wb') as f2:
    for key in SolwGE2.keys():
        f2.write("%s, %s\n" %(key, SolwGE2[key]))

### Results can be analyzed from the csv files containing the reactions and the corresponding flux values